## Organizing Steps of Recommendation system in Classes and Function

In [2]:
from pymongo import MongoClient 
import pandas as pd
import sys
import numpy as np

from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import KNNBasic


In [3]:
import datetime

In [4]:
class RecSysClass:
    
    def read(self):
        
        conn = MongoClient('192.168.1.72', 27017) 
        mydb = conn.SK
        collection1 = mydb.OrderMaster
        df1 = pd.DataFrame(list(collection1.find({},{"Skcode":1,"orderDetails":1,"_id":0})))
        return df1
    
    def ExtractItemid(self,df1):
        df1["item"] = [[i[j]['ItemId'] for j in range(len(i))] for i in df1['orderDetails']]
        df1 = df1[['Skcode','item']]
        df2 = df1.explode('item')
        return df2
        
    def Clean(self,df):
        df1 = df.drop_duplicates()
        df2 = df1.dropna()
        return df2
        
    def AddRating (self,df):
        df['rating'] = df.apply(lambda _: '1', axis=1)
        df['rating'] = df['rating'].astype(int)
        return df

    def RenameColumns(self,df):
        df.columns = ['userID','itemID', 'rating']
        return df
        
    def CalculateFrequency(self,df):
        a=df.groupby('itemID')['rating'].apply(list)
        b = a.to_frame()
        c = b.copy()
        c['frequency']  = [sum(i) for i in c['rating']]
        c1 = c.reset_index()
        c1
      
    
    def CalculateFrequency1(self):
        today = datetime.datetime.today()
        dlower = today - datetime.timedelta(days=30)

        conn = MongoClient('192.168.1.72', 27017)
        mydb = conn.SK
        collection1 = mydb.OrderMaster
        df = pd.DataFrame(list(collection1.find({"OrderDate":{'$lt': today, '$gte': dlower}},{"Skcode":1,"WarehouseId":1,"orderDetails":1,"OrderDate":1})))

        extractdf = RecSysClass().ExtractItemid(df)
        extractdf['rating'] = extractdf.apply(lambda _: '1', axis=1)
        extractdf['rating'] = extractdf['rating'].astype(int)

        a=extractdf.groupby('item')['rating'].apply(list)
        b = a.to_frame()
        c = b.copy()
        c['frequency']  = [sum(i) for i in c['rating']]
        c1 = c.reset_index()
        freq_dict = {}
        litemid = list(c1['item'])
        lfrequency = list(c1['frequency'])
        freqdict = dict(zip(litemid,lfrequency))
        return freqdict
    

    def CosineSimilarityMatrix(self,df):
        temp = []
        reader = Reader(rating_scale=(1, 5))
        data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)
        k = 10
        trainSet = data.build_full_trainset()
        sim_options = {'name': 'cosine',
                       'user_based': False
                       }
        model = KNNBasic(sim_options=sim_options)
        model.fit(trainSet)
        simsMatrix = model.compute_similarities()
        lc = list(trainSet._raw2inner_id_items)
        #df = pd.DataFrame(simsMatrix , columns = lc, index = lc)     
        ltuple = (simsMatrix,lc)
        return ltuple

    def finalfreq(self,ltup):
        lfreqfinal = {}
        lfreq = RecSysClass().CalculateFrequency1()
        ke = list(lfreq.keys())
        for i in ltup[1]:
            if (i in ke):
                lfreqfinal[i]=lfreq[i]
            else:
                lfreqfinal[i] = 1
        return lfreqfinal
    
    def insertdata(self,df):
        data_dict = df.to_dict('records')
        
        conn = MongoClient('192.168.1.101', 27017) 
        mydb = conn.ML
        collection = mydb.RecSysItem
        collection.remove() 
        collection.insert_many(data_dict)
        

In [5]:
%time df1 = RecSysClass().read()

Wall time: 39.8 s


In [6]:
%time df2 = RecSysClass().ExtractItemid(df1)

Wall time: 1.6 s


In [7]:
%time df3 = RecSysClass().Clean(df2)

Wall time: 438 ms


In [8]:
#freq = RecSysClass().CalculateFrequency1()

In [9]:
%time df4 = RecSysClass().AddRating(df3)

Wall time: 2.03 s


In [10]:
%time df5 = RecSysClass().RenameColumns(df4)

Wall time: 998 µs


In [11]:
%time ltup = RecSysClass().CosineSimilarityMatrix(df5)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Wall time: 41.4 s


In [12]:
finalfreq = RecSysClass().finalfreq(ltup)

In [13]:
SimMat = ltup[0]

In [14]:
lv = list(finalfreq.values())

In [15]:
lc = list(finalfreq.keys())

In [16]:
SimMat1 = lv * SimMat

In [22]:
df_final = pd.DataFrame(SimMat1, columns = lc , index = lc)

In [23]:
df_final

,1576,923,933,1197,1784,481,4315,3961,3960,1464,...,15990,178227,135060,329351,208517,272123,10294,225658,212850,304302
1576,1.0,3.0,1.0,4.0,2.0,1.0,1.0,1.0,1.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
923,1.0,3.0,1.0,4.0,2.0,1.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
933,1.0,3.0,1.0,4.0,2.0,1.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1197,1.0,3.0,1.0,4.0,2.0,1.0,1.0,1.0,1.0,8.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1784,1.0,3.0,1.0,4.0,2.0,1.0,1.0,1.0,1.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
10294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
225658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
212850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [24]:
df_columns = df_final.columns.astype(str)

In [26]:
lcolumn = list(df_columns)

In [28]:
df1 = pd.DataFrame(df_final.values , columns = lcolumn , index = lcolumn)

In [30]:
df1.reset_index(inplace=True,drop = True)

In [31]:
lcolumn = [int(i) for i in lcolumn]

In [33]:
df1.insert(0,"itemid",lcolumn)

In [34]:
df1

,itemid,1576,923,933,1197,1784,481,4315,3961,3960,...,15990,178227,135060,329351,208517,272123,10294,225658,212850,304302
0,1576,1.0,3.0,1.0,4.0,2.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,923,1.0,3.0,1.0,4.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,933,1.0,3.0,1.0,4.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1197,1.0,3.0,1.0,4.0,2.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1784,1.0,3.0,1.0,4.0,2.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14786,272123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
14787,10294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
14788,225658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
14789,212850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [35]:
%time RecSysClass().insertdata(df1)

C:\Users\Ankit\Anaconda3\lib\site-packages\ipykernel_launcher.py:99: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.


Wall time: 7min 57s


In [67]:
#lfreq

In [69]:
lfreqfinal[625]

15

In [36]:
v = RecSysClass().CalculateFrequency1()

In [29]:
today = datetime.datetime.today()
dlower = today - datetime.timedelta(days=30)

conn = MongoClient('192.168.1.72', 27017)
mydb = conn.SK
collection1 = mydb.OrderMaster
df = pd.DataFrame(list(collection1.find({"OrderDate":{'$lt': today, '$gte': dlower}},{"Skcode":1,"WarehouseId":1,"orderDetails":1,"OrderDate":1})))


extractdf = RecSysClass().ExtractItemid(df)
extractdf['rating'] = extractdf.apply(lambda _: '1', axis=1)
extractdf['rating'] = extractdf['rating'].astype(int)


a=extractdf.groupby('item')['rating'].apply(list)
b = a.to_frame()
c = b.copy()
c['frequency']  = [sum(i) for i in c['rating']]
c1 = c.reset_index()
freq_dict = {}
litemid = list(c1['item'])
lfrequency = list(c1['frequency'])
freqdict = dict(zip(litemid,lfrequency))


In [30]:
freqdict

{23: 1,
 138: 5,
 139: 2,
 140: 2,
 144: 2,
 145: 2,
 146: 1,
 148: 1,
 161: 1,
 162: 3,
 164: 19,
 165: 10,
 181: 6,
 182: 1,
 187: 3,
 192: 1,
 193: 1,
 195: 1,
 196: 1,
 197: 2,
 200: 2,
 201: 4,
 202: 2,
 205: 10,
 206: 4,
 207: 4,
 208: 6,
 209: 5,
 210: 4,
 416: 1,
 434: 1,
 435: 2,
 436: 3,
 445: 1,
 479: 6,
 480: 15,
 482: 13,
 483: 8,
 509: 1,
 515: 1,
 517: 1,
 519: 5,
 521: 8,
 522: 3,
 523: 3,
 526: 2,
 527: 17,
 528: 6,
 538: 5,
 540: 2,
 541: 3,
 542: 1,
 544: 7,
 545: 5,
 556: 1,
 557: 5,
 559: 1,
 572: 2,
 573: 5,
 575: 1,
 577: 1,
 585: 10,
 593: 7,
 597: 2,
 607: 1,
 622: 2,
 625: 13,
 626: 3,
 642: 2,
 644: 1,
 650: 1,
 664: 1,
 668: 2,
 669: 3,
 725: 1,
 728: 10,
 729: 4,
 730: 1,
 732: 1,
 762: 3,
 763: 3,
 764: 5,
 767: 8,
 768: 10,
 769: 3,
 772: 1,
 774: 3,
 786: 1,
 789: 1,
 795: 2,
 821: 4,
 822: 7,
 823: 1,
 824: 2,
 825: 1,
 903: 2,
 904: 4,
 905: 5,
 908: 8,
 911: 15,
 923: 2,
 926: 3,
 927: 1,
 930: 1,
 931: 5,
 932: 4,
 1008: 1,
 1106: 1,
 1107: 3,
 1108:

In [26]:
c1

,item,rating,frequency
0,23,[1],1
1,138,"[1, 1, 1, 1, 1]",5
2,139,"[1, 1]",2
3,140,"[1, 1]",2
4,144,"[1, 1]",2
...,...,...,...
4065,349681,"[1, 1]",2
4066,349848,[1],1
4067,351899,"[1, 1, 1]",3
4068,351915,"[1, 1, 1]",3


In [18]:
extractdf

,Skcode,item,rating
0,SK22543,287244,1
0,SK22543,189040,1
1,SK22543,189040,1
1,SK22543,189790,1
1,SK22543,189913,1
...,...,...,...
3246,SK16372,144421,1
3246,SK16372,231439,1
3246,SK16372,305113,1
3246,SK16372,318960,1
